1. Ask ChatGPT an SDMX question => observ the failure (the same we used in the kick-off with Daniel)
2. Explain the need for RAG [contextual_RAG](https://www.anthropic.com/news/contextual-retrieval) <- we only need a symple rag schema
3. Explain how to use openai from python
    1. explain the openai client and how to use it
    2. tokens
4. Textembedding
    - what is it & why is it needed for RAG (context size reduction)
        - talk a little bit more about tokens & vectors (king - man + woman = queen)
    - Semantic search example
5.

In [3]:
! pip install openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/13/a1/601773020ad98d6d55f45c4482ea0c30af1351827eaab98cbc7d27b4716d/openai-1.47.1-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.5.0 from https://files.pythonhosted.org/packages/9e/ef/7a4f225581a0d7886ea28359179cb861d7fbcdefad29663fc1167b86f69f/anyio-4.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/56/95/9377bcb415797e44274b51d46e3249eba641711cf3348050f76ee7b15ffc/httpx-0.27.2-py3-none-any.whl.metadata
  Obtaining dependency information for jiter<1,>=0.4.0 from https://files.pythonhosted.org/packages/03/eb/2308fa5f5c14c9

In [1]:
from IPython.display import Markdown, display
from openai import OpenAI

# Load the OpenAI API key from a file into an environment variable called OPENAI_API_KEY
%run .load_openai_api_key.py

# Instantiate the OpenAI client
client = OpenAI()

# OpenAI's Pricing
OpenAI uses a [**pay-as-you-go**](https://openai.com/api/pricing/) pricing model, which means you only pay for what you use. The costs are based on the number of tokens processed.
 - [**Tokens**](https://winder.ai/calculating-token-counts-llm-context-windows-practical-guide/) <- read this, maybe generate tokenized text with colors...
 - [Tokenizer](https://platform.openai.com/tokenizer) <- just a reference
 - [batch_mode](https://platform.openai.com/docs/guides/batch/overview) is cheapest, but using it requires advanced controlflow or offline tasks. 
   
Tell them that tokens will be explained in more detail in the next session, when we will be talking about text embeddings




In [2]:
# https://openai.com/api/pricing/
MODEL_PRICING_PER_M_TOKENS = {
    'gpt-4o': {'prompt_tokens': 5.00, 'completion_tokens': 15.00},
    'gpt-4o-2024-08-06': {'prompt_tokens': 2.50, 'completion_tokens': 10.00},
    'gpt-4o-mini': {'prompt_tokens': 0.150, 'completion_tokens': 0.600},
    'gpt-4o-mini-2024-07-18': {'prompt_tokens': 0.150, 'completion_tokens': 0.600},
    'o1-preview': {'prompt_tokens': 15.00, 'completion_tokens': 60.00}
}

PASTELS = [
      ('Pale Red', '#ffcccc')
    , ('Pale Green', '#ccffcc')
    , ('Pale Blue', '#cceeff')
    , ('Pale Yellow', '#ffffcc')
    , ('Pale Pink', '#ffccff')
    , ('Pale Orange', '#ffebcc')
    , ('Pale Purple', '#e6ccff')
    ]


# Calling the OpenAI api

### The call:
```python
response = client.chat.completions.create()
```
- client: The object representing your OpenAI client, which interfaces with the API.
- chat: The namespace for handling chat-based requests.
- completions: A sub-namespace for generating completions in a chat model.
- create(): The method that sends a request to the API to generate a response.

### Method attributes:
- model: Specifies which model to use (e.g., GPT-4).
- messages: A list of messages representing the conversation or inputs to the model.
- max_tokens: Limits the length of the model’s output (in tokens).
- temperature: Controls the randomness or creativity of the output.
- n: The number of completions (responses) the model will generate.

### 
Why choices[0] is Always Indexed at 0:  
The choices array in the API response contains all the generated completions. By default, the array has just one completion (so it's accessed via choices[0]). If you specify n>1, the API returns multiple completions, and you can loop through choices to access each completion individually.

### Most important setting
- Temperature == 1 maximizes creativity
- Temperature == 0 ensures reproducible results

In [3]:
test_message = [
        { "role": "system",
          "content": "You are a helpful assistant."},
        {
           "role": "user",
           "content": "If Sarah is older than Tom, and Tom is older than Jane, who is the youngest and how do you know?"
        }
    ]

In [4]:
# Temperature == 1 maximizes creativity, but also randomness
completion = client.chat.completions.create(
      model="gpt-4o-mini"
    , messages=test_message
    , max_tokens=100 # limits the outputted completion_tokens, so will be cut short if it exceeds max_tokens tokens
    , temperature=1  # the randomness of the output, 0 is deterministic, 1 is random (repated calls will give different results for 1)
    , n=3            # This will generate 3 separate completions
)

for i, choice in enumerate(completion.choices):
    color = PASTELS[i][1]
    display(Markdown(f"<div style='color: {color};'>{choice.message.content}</div>"))



<div style='color: #ffcccc;'>Based on the information provided: 

1. Sarah is older than Tom.
2. Tom is older than Jane.

From these two statements, we can deduce the following order of age (from oldest to youngest): Sarah > Tom > Jane.

Since Jane is the last in this order, she is the youngest. Therefore, Jane is the youngest person among the three.</div>

<div style='color: #ccffcc;'>If Sarah is older than Tom, and Tom is older than Jane, then we can conclude the following:

- Sarah > Tom (Sarah is older than Tom)
- Tom > Jane (Tom is older than Jane)

From these two statements, we can infer that Sarah is the oldest, Tom is in the middle, and Jane is the youngest.

Therefore, Jane is the youngest, as she is younger than both Tom and Sarah.</div>

<div style='color: #cceeff;'>If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. 

We can establish the age order as follows:
- Sarah > Tom (Sarah is older than Tom)
- Tom > Jane (Tom is older than Jane)

From this, we can conclude that:
- Since Sarah is older than Tom, and Tom is older than Jane, it follows that Sarah is also older than Jane.

Thus, Jane is the youngest among the three.</div>

## The exact same call, but with temperature == 0

In [5]:
# Temperature == 1 maximizes creativity, but also randomness
completion = client.chat.completions.create(
      model="gpt-4o-mini"
    , messages=test_message
    , max_tokens=100 # limits the outputted completion_tokens, so will be cut short if it exceeds max_tokens tokens
    , temperature=0  # the randomness of the output, 0 is deterministic, 1 is random (repated calls will give different results for 1)
    , n=3            # This will generate 3 separate completions
)

for i, choice in enumerate(completion.choices):
    color = PASTELS[i][1]
    display(Markdown(f"<div style='color: {color};'>{choice.message.content}</div>"))

<div style='color: #ffcccc;'>If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. 

We can determine this by analyzing the relationships:
- Sarah > Tom (Sarah is older than Tom)
- Tom > Jane (Tom is older than Jane)

From these two statements, we can infer that:
- Sarah > Tom > Jane

Since Jane is at the end of this chain, she is the youngest among the three.</div>

<div style='color: #ccffcc;'>If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. 

We can determine this by analyzing the relationships:
- Sarah > Tom (Sarah is older than Tom)
- Tom > Jane (Tom is older than Jane)

From these two statements, we can infer that:
- Sarah > Tom > Jane

Since Jane is at the end of this chain, she is the youngest among the three.</div>

<div style='color: #cceeff;'>If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. 

We can determine this by analyzing the relationships:
- Sarah > Tom (Sarah is older than Tom)
- Tom > Jane (Tom is older than Jane)

From these two statements, we can infer that:
- Sarah > Tom > Jane

Since Jane is at the end of this chain, she is the youngest among the three.</div>

In [1]:
from openai import OpenAI

# Load the OpenAI API key from a file into an environment variable called OPENAI_API_KEY
%run .load_openai_api_key.py

# Instantiate the OpenAI client
client = OpenAI()

In [2]:
MODEL_PRICING_PER_M_TOKENS = {
    'gpt-4o': {'prompt_tokens': 5.00, 'completion_tokens': 15.00},
    'gpt-4o-2024-08-06': {'prompt_tokens': 2.50, 'completion_tokens': 10.00},
    'gpt-4o-mini': {'prompt_tokens': 0.150, 'completion_tokens': 0.600},
    'gpt-4o-mini-2024-07-18': {'prompt_tokens': 0.150, 'completion_tokens': 0.600},
    'o1-preview': {'prompt_tokens': 15.00, 'completion_tokens': 60.00}
}

def model(persona, prompt, model="gpt-4o-mini"):
    completion = client.chat.completions.create(
          model=model
        , messages=[
            { "role": "system", "content": persona},
            { "role": "user", "content": prompt}
    ]
        , temperature=0
    )
    # Get the pricing for the model used in the completion
    pricing = MODEL_PRICING_PER_M_TOKENS[completion.model]

    # Calculate the cost of the completion
    prompt_cost = completion.usage.prompt_tokens * pricing['prompt_tokens']
    generation_cost = completion.usage.completion_tokens * pricing['completion_tokens']
    total_cost = (prompt_cost + generation_cost) / 10**6

    # Extract the message from the completion
    message = completion.choices[0].message.content

    return message, total_cost

In [3]:
model("you are a helpful assistant", "What is the capital of France?")

('The capital of France is Paris.', 7.65e-06)

In [7]:
model("you are a helpful assistant", "What is the capital of France?", model='gpt-4o-2024-08-06')

('The capital of France is Paris.', 0.0001275)

# The parsing completion object


In [27]:
dict(completion)

{'id': 'chatcmpl-ABdLV8se4vnhnwiZ1WEAhGy7zvDpt',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. \n\nWe can determine this by analyzing the relationships:\n- Sarah > Tom (Sarah is older than Tom)\n- Tom > Jane (Tom is older than Jane)\n\nFrom these two statements, we can infer that:\n- Sarah > Tom > Jane\n\nSince Jane is at the end of this chain, she is the youngest among the three.', refusal=None, role='assistant', function_call=None, tool_calls=None)),
  Choice(finish_reason='stop', index=1, logprobs=None, message=ChatCompletionMessage(content='If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. \n\nWe can determine this by analyzing the relationships:\n- Sarah > Tom (Sarah is older than Tom)\n- Tom > Jane (Tom is older than Jane)\n\nFrom these two statements, we can infer that:\n- Sarah > Tom > Jane\n\nSince Jane i

In [6]:
def parse_openai_completion_response(completion, print_cost=True):
    """

    """
    
    # Get the pricing for the model used in the completion
    pricing = MODEL_PRICING_PER_M_TOKENS[completion.model]

    # Calculate the cost of the completion
    prompt_cost = completion.usage.prompt_tokens * pricing['prompt_tokens']
    generation_cost = completion.usage.completion_tokens * pricing['completion_tokens']
    total_cost = (prompt_cost + generation_cost) / 10**6

    # Extract the message from the completion
    message = completion.choices[0].message.content

    # Print the cost of the completion
    if print_cost:
        print('Cost: cent {:.3f}'.format(total_cost * 100))
        print('Cost: USD {:.5f}'.format(total_cost))

    return message


In [7]:
ans = parse_openai_completion_response(completion)

Cost: cent 0.016
Cost: USD 0.00016


In [9]:
ans = parse_openai_completion_response(completion, print_cost=False)

In [10]:
print(ans)

If Sarah is older than Tom, and Tom is older than Jane, then Jane is the youngest. 

We can determine this by analyzing the relationships:
- Sarah > Tom (Sarah is older than Tom)
- Tom > Jane (Tom is older than Jane)

From these two statements, we can infer that:
- Sarah > Tom > Jane

Since Jane is at the end of this chain, she is the youngest among the three.


In [11]:
len(completion.choices)

3